# Notebook to explore and visualize neighborhoods

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library
import urllib.request
from bs4 import BeautifulSoup

print('Import done')

Import done


In [2]:
### Shortened code to create dataframe and append coordinates as in part 1 and 2 but without outputs

In [3]:
# URL/web page to be scraped to get ZIP codes of Canada
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")
right_table=soup.find('table', class_='wikitable sortable')

# Extract postal codes and corresponding boroughs and neighborhoods and create the desired pandas dataframe
Postcode=[]
Borough=[]
Neighborhood=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        Postcode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighborhood.append(cells[2].find(text=True))
        
CA_postcodes = pd.DataFrame({'PostalCode': Postcode, 'Borough': Borough, 'Neighborhood': Neighborhood})

# Identify and drop entries with not assigned boroughs (and delete '\n'-extensions)
index_Notassigned = CA_postcodes[CA_postcodes['Borough'] == 'Not assigned'].index
CA_postcodes.drop(index_Notassigned, inplace=True)
CA_postcodes = CA_postcodes.replace('\n','', regex=True)

# Identify not assigned neighborhoods and replace 'Not assigned' with the corresponding borough
i = 0
for checker in CA_postcodes['Neighborhood']:
    i = i + 1
    if checker == 'Not assigned':
        CA_postcodes['Neighborhood'][i+1] = CA_postcodes['Borough'][i+1]
        
# Group neighborhoods to a single postal code and use the .shape-method to print the dimensions of the dataframe
final_df = CA_postcodes.groupby(['PostalCode','Borough'], sort=False)['Neighborhood'].apply(', '.join).to_frame(name = 'Neighborhood').reset_index()

print('The dataframe has',final_df.shape[0],'rows and',final_df.shape[1],'columns')
final_df.head(12)

The dataframe has 103 rows and 3 columns


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [4]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [5]:
import geocoder
# add latitude and longitude to pandas dataframe
final_df['Latitude'] = ""
final_df['Longitude'] = ""
cnt = 10

for i in range(len(final_df)):
    lat_lng_coords = None
    postal_code = final_df['PostalCode'][i]
    # loop until you get the coordinates
    while(lat_lng_coords is None):

        #g = geocoder.google('{M4B}, Toronto, Ontario'.format(postal_code))
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        final_df['Latitude'][i] = lat_lng_coords[0]
        final_df['Longitude'][i] = lat_lng_coords[1]
        
        # Progress checker for every ten updated entries
        if i % cnt == 0:
            print('Coordinates appended for',i,'of',len(final_df),'entries')        

final_df.head(12)

Coordinates appended for 0 of 103 entries
Coordinates appended for 10 of 103 entries
Coordinates appended for 20 of 103 entries
Coordinates appended for 30 of 103 entries
Coordinates appended for 40 of 103 entries
Coordinates appended for 50 of 103 entries
Coordinates appended for 60 of 103 entries
Coordinates appended for 70 of 103 entries
Coordinates appended for 80 of 103 entries
Coordinates appended for 90 of 103 entries
Coordinates appended for 100 of 103 entries


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7524,-79.3293
1,M4A,North York,Victoria Village,43.7304,-79.3133
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6551,-79.3626
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7231,-79.4516
4,M7A,Queen's Park,Queen's Park,43.6611,-79.391
5,M9A,Etobicoke,Islington Avenue,43.6622,-79.5284
6,M1B,Scarborough,"Rouge, Malvern",43.8115,-79.1955
7,M3B,North York,Don Mills North,43.7492,-79.3619
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7075,-79.3118
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6574,-79.3782


### Create a map with latitude and longitude of 'postal code coordinates'

In [6]:
toronto_data = final_df
# create map of Toronto using latitude and longitude values
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.5)

# add markers to map
for lat, lng, borough, neighborhood in zip(final_df['Latitude'], final_df['Longitude'], final_df['Borough'], final_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

#### Define Foursquare Credentials and Version

##### I have created a json-file named 'secrets.json' with my credential in the form:
##### {
#####     "CLIENT_ID": "your_client_id",
#####     "CLIENT_SECRET": "your_client_secret",
#####     "VERSION": "your_version"
##### }
##### This has been saved in the directory of the notebook. To proceed, please create a similar json-file 
##### as described above with your credential and proceed with the following cell calling the credentials:

In [9]:
secrets = json.load(open('secrets.json'))
CLIENT_ID = secrets['CLIENT_ID']
CLIENT_SECRET = secrets['CLIENT_SECRET']
VERSION = secrets['VERSION']

### Getting categories with the **get_category_type** function from the Foursquare lab.

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## As in the example for NYC we explore the Neighborhoods in Toronto

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Use the getNearbyVenues-function on each neighborhood and create a the dataframe 'toronto_venues'.

In [14]:
limit = 100
radius = 500

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

print(toronto_venues.shape)
toronto_venues.head()

(2481, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752440,-79.329271,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752440,-79.329271,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.752440,-79.329271,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.730421,-79.313320,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.730421,-79.313320,Memories of Africa,43.726602,-79.312427,Grocery Store


#### Check the number of venues for each neighborhood (postal code)

In [15]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,13,13,13,13,13,13
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",14,14,14,14,14,14
"Alderwood, Long Branch",4,4,4,4,4,4
Bayview Village,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Berczy Park,66,66,66,66,66,66
"Birch Cliff, Cliffside West",6,6,6,6,6,6


#### Number of unique categories from all returned venues

In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 259 uniques categories.


## Analyze Each Neighborhood (postal code)

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe and move it to the first column
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# Neighborhood column to be moved to the first column
idx_nghbr = toronto_onehot.columns.get_loc("Neighborhood")

fixed_columns = [toronto_onehot.columns[idx_nghbr]] + list(toronto_onehot.columns[0:idx_nghbr]) + list(toronto_onehot.columns[idx_nghbr+1:len(toronto_onehot.columns)])
toronto_onehot = toronto_onehot[fixed_columns]

print('New size of the dataframe:',toronto_onehot.shape)
toronto_onehot.head()

New size of the dataframe: (2481, 259)


,Neighborhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Group rows by neighborhood (postal code) and by taking the mean of the frequency of occurrence of each category

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

print('Confirmation of the adapted size; new size:', toronto_grouped.shape)
toronto_grouped.head()

Confirmation of the adapted size; new size: (100, 259)


,Neighborhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Quad,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.03,0.0,0.01,0.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.030000,0.0,0.03,0.0,0.0,0.0,0.01,0.000000,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.01,0.02,0.0,0.0,0.0,0.0,0.01,0.03,0.01,0.0,0.0,0.0,0.0,0.06,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,

#### Output of the top 5 most common venues for each neighborhood

In [19]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
           venue  freq
0    Coffee Shop  0.09
1           Café  0.06
2          Hotel  0.05
3  Deli / Bodega  0.03
4   Burger Joint  0.03


----Agincourt----
                venue  freq
0  Chinese Restaurant  0.15
1       Shopping Mall  0.15
2         Supermarket  0.15
3      Discount Store  0.08
4    Sushi Restaurant  0.08


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
               venue  freq
0           Pharmacy   1.0
1  Afghan Restaurant   0.0
2      Moving Target   0.0
3     Massage Studio   0.0
4     Mattress Store   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.14
1          Pizza Place  0.07
2  Fried Chicken Joint  0.07
3     Sushi Restaurant  0.07
4          Coffee Shop  0.07


----Alderwood, Long Branch----
               venue  freq
0  Convenience Store  0.25
1                Gym  0.25
2   

function to sort the venues in descending order.

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Bakery,Bar,Japanese Restaurant,Asian Restaurant,Burger Joint,Restaurant
1,Agincourt,Shopping Mall,Chinese Restaurant,Supermarket,Discount Store,Hong Kong Restaurant,Bakery,Pool,Sushi Restaurant,Shanghai Restaurant,Vietnamese Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Concert Hall,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pharmacy,Coffee Shop,Park,Liquor Store,Sandwich Place,Fried Chicken Joint,Beer Store,Sushi Restaurant,Japanese Restaurant
4,"Alderwood, Long Branch",Convenience Store,Sandwich Place,Gym,Pub,Yoga Studio,Elementary School,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


## 4. Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [23]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0, init='k-means++').fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 0, 0, 3, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [24]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged['Cluster Labels'].fillna(max(toronto_merged['Cluster Labels'])+1, inplace=True)   # unlabeled entries
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype('int64')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M3A,North York,Parkwoods,43.7524,-79.3293,Food & Drink Shop,Park,Fast Food Restaurant,Yoga Studio,Dog Run,Field,Farmers Market,Farm,Falafel Restaurant,Event Space,3
1,M4A,North York,Victoria Village,43.7304,-79.3133,Grocery Store,French Restaurant,Park,Intersection,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Yoga Studio,3
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6551,-79.3626,Coffee Shop,Gym / Fitness Center,Restaurant,Breakfast Spot,Yoga Studio,Thai Restaurant,Italian Restaurant,Food Truck,Event Space,Electronics Store,0
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7231,-79.4516,Clothing Store,Dessert Shop,Cosmetics Shop,Toy / Game Store,Chocolate Shop,Men's Store,Food Court,Furniture / Home Store,Pharmacy,Electronics Store,0
4,M7A,Queen's Park,Queen's Park,43.6611,-79.391,Coffee Shop,Sandwich Place,Café,Bubble Tea Shop,Italian Restaurant,Indian Restaurant,Food Truck,Bookstore,Juice Bar,Smoothie Shop,0


In [25]:
toronto_merged['Cluster Labels'].value_counts().sort_index()

0    75
1     1
2     1
3    19
4     4
5     3
Name: Cluster Labels, dtype: int64

Finally, let's visualize the resulting clusters

In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

### Write down the number of cluster members for each cluster

In [27]:
toronto_merged['Cluster Labels'].value_counts().sort_index()

0    75
1     1
2     1
3    19
4     4
5     3
Name: Cluster Labels, dtype: int64

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1 - Food & Drinks

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
2,Downtown Toronto,Coffee Shop,Gym / Fitness Center,Restaurant,Breakfast Spot,Yoga Studio,Thai Restaurant,Italian Restaurant,Food Truck,Event Space,Electronics Store,0
3,North York,Clothing Store,Dessert Shop,Cosmetics Shop,Toy / Game Store,Chocolate Shop,Men's Store,Food Court,Furniture / Home Store,Pharmacy,Electronics Store,0
4,Queen's Park,Coffee Shop,Sandwich Place,Café,Bubble Tea Shop,Italian Restaurant,Indian Restaurant,Food Truck,Bookstore,Juice Bar,Smoothie Shop,0
6,Scarborough,Pet Store,Home Service,Yoga Studio,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Farm,0
8,East York,Fast Food Restaurant,Pizza Place,Pharmacy,Athletics & Sports,Rock Climbing Spot,Gastropub,Intersection,Pub,Bank,Breakfast Spot,0
9,Downtown Toronto,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Fast Food Restaurant,Diner,Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Lingerie Store,0
10,North York,Pizza Place,Fast Food Restaurant,Asian Restaurant,Sushi Restaurant,Rental Car Location,Latin American Restaurant,Mediterranean Restaurant,Japanese Restaurant,Grocery Store,Fish Market,0
11,Etobicoke,Pizza Place,Bank,Tea Room,Sandwich Place,Chinese Restaurant,Electronics Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,0
12,Scarborough,Golf Course,Bar,Yoga Studio,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Event Space,0
13,North York,Grocery Store,Beer Store,Smoke Shop,Intersection,Gym,Coffee Shop,Ethiopian Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,0


#### Cluster 2 - Entertainment

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
45,North York,Music Venue,Yoga Studio,Dog Run,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,1


#### Cluster 3 - Car

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
71,Scarborough,Auto Garage,Yoga Studio,Event Space,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Dog Run,2


#### Cluster 4 - Park and spare time

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,North York,Food & Drink Shop,Park,Fast Food Restaurant,Yoga Studio,Dog Run,Field,Farmers Market,Farm,Falafel Restaurant,Event Space,3
1,North York,Grocery Store,French Restaurant,Park,Intersection,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Yoga Studio,3
7,North York,Coffee Shop,Park,Burger Joint,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,3
18,Scarborough,Construction & Landscaping,Park,Gym / Fitness Center,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Falafel Restaurant,3
22,Scarborough,Business Service,Park,Korean Restaurant,Coffee Shop,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Yoga Studio,3
35,East York,Italian Restaurant,Park,Farmers Market,Bar,Ethiopian Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,3
39,North York,Construction & Landscaping,Park,Golf Driving Range,Trail,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,3
41,East Toronto,Grocery Store,Fast Food Restaurant,Park,Discount Store,Bus Line,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,3
46,North York,Park,Mobile Phone Shop,Yoga Studio,Dog Run,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,3
49,North York,Trail,Bakery,Basketball Court,Park,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,3


#### Cluster 5 - Medics

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Etobicoke,4,Pharmacy,Baseball Field,Café,Park,Skating Rink
64,York,4,Pizza Place,Convenience Store,Fried Chicken Joint,Diner,Pharmacy
82,Scarborough,4,Pharmacy,Pizza Place,Hobby Shop,Coffee Shop,Fried Chicken Joint
85,Scarborough,4,Pharmacy,Concert Hall,Fish & Chips Shop,Field,Fast Food Restaurant
